In [1]:
import pandas as pd
import ccxt
from importlib import import_module

ccxt_modules = [mod for mod in dir(ccxt) if mod[0].islower() and mod not in 
                ['base','decimal_to_precision','error_hierarchy',
                 'errors','exchanges','static_dependencies']]
rows = []
for mod in ccxt_modules:
    full_module_name = "ccxt." + mod

    imp_module = import_module(full_module_name)
    describe = getattr(imp_module, mod)().describe()
    api_elem = describe['urls']['api']
    if isinstance(api_elem, str):
        api_url = api_elem
    else:
        assert isinstance(api_elem, dict)
        if 'private' in api_elem.keys():
            api_url = api_elem['private']
        elif 'v3Private' in api_elem.keys():
            api_url = api_elem['v3Private']
        elif 'trade' in api_elem.keys():
            api_url = api_elem['trade']
        elif 'api' in api_elem.keys():
            api_url = api_elem['api']
        elif 'public' in api_elem.keys():
            api_url = api_elem['public']
        elif 'rest' in api_elem.keys():
            api_url = api_elem['rest']
        elif 'publicV2' in api_elem.keys():
            api_url = api_elem['publicV2']
        elif 'current' in api_elem.keys():
            api_url = api_elem['current']
        elif len(api_elem) == 1:
            api_url = api_elem[api_elem.keys()[0]]
        else:
            print("Unexpected json structure")
    if 'hostname' in api_url:
        api_url = api_url.format(hostname=describe['hostname'])

    rows.append([describe['id'], api_url])

exchanges_df = pd.DataFrame(rows, columns=['name','api_url']).set_index('name')
exchanges_df

,api_url
name,
aax,https://api.aax.com
acx,https://acx.io/api
aofex,https://openapi.aofex.com/openApi
bequant,https://api.bequant.io
bibox,https://api.bibox365.com
...,...
xbtce,https://cryptottlivewebapi.xbtce.net:8443/api
xena,https://api.xena.exchange
yobit,https://yobit.net/tapi


In [5]:
exchanges_df = exchanges_df.merge(aws_ip_ranges_df, how='left', left_on='api_asn', right_index=True)
exchanges_df

,api_url,api_url_domain,api_ip,api_asn,region,service,network_border_group
name,,,,,,,
aax,https://api.aax.com,api.aax.com,52.220.66.226,52.220.0.0/15,ap-southeast-1,AMAZON,ap-southeast-1
aax,https://api.aax.com,api.aax.com,52.220.66.226,52.220.0.0/15,ap-southeast-1,EC2,ap-southeast-1
acx,https://acx.io/api,acx.io,3.106.77.227,3.104.0.0/14,ap-southeast-2,AMAZON,ap-southeast-2
acx,https://acx.io/api,acx.io,3.106.77.227,3.104.0.0/14,ap-southeast-2,EC2,ap-southeast-2
aofex,https://openapi.aofex.com/openApi,openapi.aofex.com,104.20.161.135,104.20.160.0/20,NaN,NaN,NaN
...,...,...,...,...,...,...,...
xena,https://api.xena.exchange,api.xena.exchange,104.22.73.120,104.22.64.0/20,NaN,NaN,NaN
yobit,https://yobit.net/tapi,yobit.net,104.16.241.98,104.16.240.0/20,NaN,NaN,NaN
zaif,https://api.zaif.jp,api.zaif.jp,13.112.252.96,13.112.0.0/14,ap-northeast-1,AMAZON,ap-northeast-1
